# IBM  Db2 Event Store Lab

<p>This code provides some basic manipulation of the Event Store database.</p>

## 1.  Connect to IBM Db2 Event Store
To establish a connection to IBM Db2 Event Store, you need connection endpoints. Use the configuration reader to provide a set of APIs for Event Store connection and configuration.<br/>
`# ConfigurationReader.setConnectionEndpoints("<HostName>:<PortNumber>")`

You can also specify multiple connection endpoints by providing a connection string that contains comma-separated list of HostName:PortNumber pairs.<br/>
`# ConfigurationReader.setConnectionEndpoints("<HostName1>:<PortNumber1>,<HostName2>:<PortNumber2>,<Hostname3>:<PortNumber3>")` 

Using the configuration reader API, set up the userID and password that will be used to connect to Event Store.

Note that the `ConnectionEndpoint` IP address should be the same as in your browser URL.

In [ ]:
from eventstore.oltp import EventContext
from eventstore.oltp.row_generator import generate_tele
from eventstore.catalog import TableSchema, IndexSpecification, SortSpecification, ColumnOrder
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from eventstore.common import ConfigurationReader

spark = SparkSession.builder.getOrCreate()
ConfigurationReader.setConnectionEndpoints("173.19.0.1:1100") # This should be the same IP address as this browser page
ConfigurationReader.setEventUser("admin")
ConfigurationReader.setEventPassword("password")
spark.version

## 2. Define a database  
Only one database can be active in Db2 Event Store. If you already have a database, you don't need to create one.<br/>
To create a database in Event Store, you can use the `createDatabase` method 

In [ ]:
EventContext.drop_database("TESTDB") # uncomment this line if you need to first drop the database
EventContext.create_database("TESTDB")

###  2.1 Open an existing database
To use an existing database, use the following call:

In [ ]:
ctx = EventContext.get_event_context("TESTDB")

<a id="create-table"></a>
## 3. Create your table

### 3.1 Define a schema for the table
To create a new table, you must first specify a schema for the table.
Specify the columns, sharding key, and primary key, as required.<br/>
Note that currently String is not supported as a primary key or sharding key.<br/>
<i>The `timestamp` is not supported in the first release but still works</i>

In [ ]:
from eventstore.catalog import TableSchema
from pyspark.sql.types import *

schema = StructType([
        StructField("id", LongType(), nullable = False),
        StructField("taxiId", StringType(), nullable = True),
        StructField("tripId", StringType(), nullable = True),
        StructField("tripStartTime", TimestampType(), nullable = True),
        StructField("tripEndTime", TimestampType(), nullable = True),
        StructField("pickupLong", DoubleType(), nullable = True),
        StructField("pickupLat", DoubleType(), nullable = True),
        StructField("dropoffLong", DoubleType(), nullable = True),
        StructField("dropoffLat", DoubleType(), nullable = True),
        StructField("tripTotal", DoubleType(), nullable = True),
        StructField("tripMiles", DoubleType(), nullable = True),
        StructField("tripSeconds", DoubleType(), nullable = True)
])  
table_schema = TableSchema("TaxiTrips", schema,
                            sharding_columns=["id"],
                            pk_columns=["id"])

<b>Tip:</b> Databases in Event Store are partitioned into shards. Any Event Store node of a multi-node Event Store cluster contains 0, 1 or N shards of the defined database. In addition to the mandatory shard key, there is also the option to provide a primary key. When this key is defined, Event Store ensures that only a single version of each primary key exists in the database.

In the above example, a sharding key and a primary key are defined on column id. 

<a id="create-table-two"></a>
### 3.2 Create the table
Create the Event Store table based on the above, unresolved schema.

In [ ]:
ctx.create_table(table_schema)
table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
    print(name)

### 3.3 Start the insertion program
At this point we need to start the insertion program.<br/>
Once started, it would be interesting to go see what is shown iun the IBM Data Platform Manager.

## 4. Query the table 

<a id="connect-to-es-two"></a>
### 4.1 Connect to  Event Store 

When connecting Event Store for the first time, you need to provide connection endpoints using the configuration reader.<br/>
If you are in the same session that created the database and table, you don't need to execute this step.<br/>
If you are in a new session, make sure you executed step 1.

### 4.2 Create EventSession

To run a Spark SQL query, you need to establish an Event Store Spark session.

In [ ]:
from eventstore.sql import EventSession

eventSession = EventSession(spark.sparkContext, "TESTDB")
eventSession.open_database()
eventSession.set_query_read_option("SnapshotNone")

### 4.3 Prepare a DataFrame for the query 
The following API provides a DataFrame that holds the query results on the Event Store table. 

In [ ]:
reviewTab = eventSession.load_event_table("TaxiTrips")
reviewTab.createOrReplaceTempView("TaxiTrips")

### 4.4 Run the SQL query
Now you can materialize the dataframe associated with the sql query by using show().

In [ ]:
resultSet1 = eventSession.sql("SELECT count(*) as totalRows FROM TaxiTrips")
resultSet1.show()

This query shows an aggregation. Since it is executed after the previous one you should see a larger number of total records

In [ ]:
resultSet2 = eventSession.sql("""
SELECT date_format(tripStartTime, 'HH:mm') as StartTime, count(*) as count, avg(tripTotal) as AvgTotal,
                      avg(tripMiles) as AvgMiles, avg(tripSeconds) as AvgDuration
FROM TaxiTrips
GROUP BY date_format(tripStartTime, 'HH:mm')
ORDER BY 1
""")
resultSet2.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import time

for i in range(0, 100):  # Loop 100 times if empty
    if not resultSet2.head(1):
        if i == 0:
            print("Empty dataset. Start the event loader.")
        print('.', end='')
    else:
        print("Show the SQL results (and repeat to show updates):")
        break

for i in range(0, 3):
    resultSet2 = eventSession.sql("""
SELECT date_format(tripStartTime, 'HH:mm') as StartTime, count(*) as count, avg(tripTotal) as AvgTotal,
                      avg(tripMiles) as AvgMiles, avg(tripSeconds) as AvgDuration
FROM TaxiTrips
GROUP BY date_format(tripStartTime, 'HH:mm')
ORDER BY 1
""")
    resultSet2.show()
    time.sleep(1);

In [ ]:
import matplotlib.cm as cm

colors = cm.Set3(np.linspace(0,1,48))
plt.rcParams["figure.figsize"] = (15, 3)

resultSet2 = eventSession.sql("""
SELECT date_format(tripStartTime, 'HH:mm') as StartTime, count(*) as count, avg(tripTotal) as AvgTotal,
                      avg(tripMiles) as AvgMiles, avg(tripSeconds) as AvgDuration
FROM TaxiTrips
GROUP BY date_format(tripStartTime, 'HH:mm')
ORDER BY 1
""")

pdf = resultSet2.toPandas();
ax = pdf[['StartTime', 'count']].plot(x='StartTime', y='count', kind='bar', color=colors, legend=None)
ax2 = ax.twinx()
pdf[['StartTime', 'AvgDuration']].plot(x='StartTime', y='AvgDuration', linestyle='-', marker='o', ax=ax2)

ax2.set_ylabel("Trip Count")
ax.set_ylabel("Avg Duration")
ax.set_xlabel("Start Time")
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 );


### You can stop the loader program with the following cell
Otherwise, it automatically stops after 30 minutes

In [ ]:
# # Stop the loader program
# s = socket.socket()
# s.connect((host, port))
# msg = "stop\n"
# s.send(msg.encode())
# ret = s.recv(1024).decode()
# s.close
# print(ret)